In [ ]:
import numpy as np
from scipy import stats
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox
import plotly.graph_objects as go
import pickle as pkl
import keras
import tensorflow_addons as tfa
import os
from GraphFuncsV4 import*
import time
from tensorflow.keras.layers import *
import keras.backend as K

In [ ]:
#Import model
modelname = 'ConvV1_1.0e-03_25_32_2'
model = keras.models.load_model('./results/%s.h5' % modelname)

In [ ]:
#Create folders for plots
try:
    os.mkdir("./Plots/%s" % modelname)
    os.mkdir("./Plots/%s/Model" % modelname)
    os.mkdir("./Plots/%s/Model/Single Model" % modelname)
    os.mkdir("./Plots/%s/Comparison" % modelname)
    os.mkdir("./Plots/%s/Comparison/Single Model" % modelname)
    os.mkdir("./Plots/%s/Analytical" % modelname)
    os.mkdir("./Plots/%s/Analytical/Single Model" % modelname)
except OSError:
    print("Folder %s already exists" % modelname)

In [ ]:
#Import scaler and Monte Carlo validation data
savedir = "./results/"
yscaler = pkl.load(open(savedir + "yscaler", "rb"))
testin = pkl.load(open(savedir + "testin", "rb"))
testout = pkl.load(open(savedir + "testout", "rb"))

In [ ]:
#Make model predictions and test inference time of model
true = testout.numpy()
seconds = time.time()
print("Start time:", seconds)
pred = model.predict(testin) #make model predictions
seconds2 = time.time()
print("End time:", seconds2)
print("Images:", len(testin.numpy()))
true = yscaler.inverse_transform(true)
true *= 2 #Scale by 2 for conversion from defocus to micrometers (approximately a factor of 2)
pred = yscaler.inverse_transform(pred)
pred *= 2 #Scale by 2 for conversion from defocus to micrometers (approximately a factor of 2)
print("Images/second:", len(testin.numpy())/(seconds2-seconds))

In [ ]:
#Variable(s) of interest
varname = [
#            "Amplitude", 
#            "Decay Coefficient", 
#            "Spherical Aberration", 
#            "Astigmatism", 
#            "Principal Axis Angle", 
           "Defocus", 
#            "Detuning"
           ]

In [ ]:
print(type(pred[0][0]))

In [ ]:
bins = 10 #Number of bins for plots
space = 'Single Model' #Name of sub-folder

#Generate plots
for i in range(len(varname)):
    lower, upper = getlowerupper(pred[:,i], true[:,i])
    
    preds = np.array([pred[:,i]])
    
    prednames = ['ML Model']
    
    heatmap(modelname, 
            'Model', 
            space, 
            true[:,i], 
            pred[:,i], 
            varname[i], 
            lower, 
            upper, 
            bins)
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model', 
                 space, 
                 true[:,i], 
                 pred[:,i], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)
    
    confinttable(modelname, 
                 space, 
                 'Model', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)